In [1]:
import torch
print("CUDA 是否可用：", torch.cuda.is_available())
print("GPU 型號：", torch.cuda.get_device_name(0))


CUDA 是否可用： True
GPU 型號： NVIDIA GeForce RTX 3060


## 選擇其中一個Model
### Speech To Text, STT 
- small

- medium

- large-v2

In [ ]:
import whisper

# 加載 Whisper 並讓模型運行在 GPU
model = whisper.load_model("small").to("cuda")

# 轉錄音檔
file_path = r"C:\Users\User\Dropbox\May4s\employee_log.m4a"
transcription = model.transcribe(file_path)

# 輸出轉錄結果
print(transcription["text"])


c:\Users\User\anaconda3\envs\may4s\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


轉錄內容： 我是員工編號EMP123 今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成的模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LN推理速度嘗試用量化技術減少記憶體的使用


In [21]:
import whisper

# 加載 Whisper 並讓模型運行在 GPU
model = whisper.load_model("medium").to("cuda")

# 轉錄音檔
file_path = r"C:\Users\User\Dropbox\May4s\employee_log.m4a"
transcription = model.transcribe(file_path)

# 輸出轉錄結果
print(transcription["text"])


100%|█████████████████████████████████████| 1.42G/1.42G [01:51<00:00, 13.7MiB/s]


我是員工編號EMP123,今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫,今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足,明天計畫優化LLM退理速度嘗試用量化技術減少記憶體的使用


In [2]:
import whisper

# 加載 Whisper 並讓模型運行在 GPU
model = whisper.load_model("large-v2").to("cuda")

# 轉錄音檔
file_path = r"C:\Users\User\Dropbox\May4s\employee_log.m4a"
transcription = model.transcribe(file_path)

# 輸出轉錄結果
print(transcription["text"])


100%|█████████████████████████████████████| 2.87G/2.87G [03:44<00:00, 13.8MiB/s]


我是員工編號EMP123今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LLM推理速度嘗試用量化技術減少記憶體的使用


## 呼叫 Ollama API

### mistral

In [32]:
import requests
import json
import re
# 連接到 Ollama 伺服器
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt):
    payload = {
        "model": "mistral:latest",
        "prompt": prompt,
        "stream": False  # 設為 False 讓回應一次返回
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

# 產生 Prompt
prompt = f"""
請根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記，並輸出成 JSON 格式：
---
{transcription["text"]}
---
請以 JSON 格式輸出，包含：
- "employee_id"
- "timestamp"
- "project"
- "summary"
"""

# 呼叫 Ollama
output_text = query_ollama(prompt)
print("LLM 輸出結果：", output_text)


# 解析 JSON，去除 LLM 可能產生的多餘文字
match = re.search(r"\{.*\}|\[.*\]", output_text, re.DOTALL)
if match:
    json_text = match.group(0)  # 提取純 JSON
    try:
        parsed_output = json.loads(json_text)
        print("✅ 解析成功：", parsed_output)
    except json.JSONDecodeError as e:
        print("❌ JSON 解析錯誤：", e)
else:
    print("❌ 解析失敗，請檢查 LLM 輸出")


LLM 輸出結果：  {
      "employee_id": "EMP123",
      "timestamp": "2025-02-11T10:00:00",
      "project": "AI研發計畫",
      "summary": "完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足,明天計畫優化LLM退理速度嘗試用量化技術減少記憶體的使用"
   }
✅ 解析成功： {'employee_id': 'EMP123', 'timestamp': '2025-02-11T10:00:00', 'project': 'AI研發計畫', 'summary': '完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足,明天計畫優化LLM退理速度嘗試用量化技術減少記憶體的使用'}


In [33]:
import pandas as pd

# 轉成 DataFrame
df = pd.DataFrame([parsed_output])

# 顯示表格
print(df)

# 存成 CSV
df.to_csv("employee_log.csv", index=False)


  employee_id            timestamp project  \
0      EMP123  2025-02-11T10:00:00  AI研發計畫   

                                             summary  
0  完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足,明天計畫優化LLM退理速度嘗試用量化...  


### deepseek-r1:8b

In [34]:
import requests
import json

# 連接到 Ollama 伺服器
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt):
    payload = {
        "model": "deepseek-r1:8b",
        "prompt": prompt,
        "stream": False  # 設為 False 讓回應一次返回
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

# 產生 Prompt
prompt = f"""
請根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記，並輸出成 JSON 格式：
---
{transcription["text"]}
---
請以 JSON 格式輸出，包含：
- "employee_id"
- "timestamp"
- "project"
- "summary"
"""

# 呼叫 Ollama
output_text = query_ollama(prompt)
print("LLM 輸出結果：", output_text)

# 解析 JSON，去除 LLM 可能產生的多餘文字
match = re.search(r"\{.*\}|\[.*\]", output_text, re.DOTALL)
if match:
    json_text = match.group(0)  # 提取純 JSON
    try:
        parsed_output = json.loads(json_text)
        print("✅ 解析成功：", parsed_output)
    except json.JSONDecodeError as e:
        print("❌ JSON 解析錯誤：", e)
else:
    print("❌ 解析失敗，請檢查 LLM 輸出")



LLM 輸出結果： <think>
今天收到一个用户的查询，他提供了一段员工日志，让我帮他解析出专案名称、员工ID和时间戳，并以JSON格式输出。看起来用户可能是一位需要处理这些数据的管理人员或者数据分析师。

首先，我要理解用户的需求。他给了一个例子，里面有员工编号EMP123，日期是2025年2月11日早上10点，项目名称是AI研發計畫，完成了一些工作，但遇到了GPU内存不足的问题，明天计划优化LLM训练速度。然后他要求生成一个包含employee_id、timestamp、project和summary的JSON。

接下来，我需要分析用户提供的日志内容，提取这些信息。员工ID很明显是EMP123，时间戳是2025年2月11日早上10点，可以转换成ISO格式。项目名称就是AI研發計畫。摘要部分则需要总结当天的工作和问题，比如完成模型训练和部署测试，但遇到GPU内存不足的问题，明天计划进行优化。

用户可能希望自动化处理这些日志数据，所以他们提供了一个例子，看起来是希望生成类似的JSON格式。因此，我需要确保解析正确，并且结构符合要求。

最后，检查是否有其他潜在需求，比如时间戳的格式转换或者其他字段的可能性，但根据用户的要求，只需要提取这四个部分。
</think>

{
  "employee_id": "EMP123",
  "timestamp": "2025-02-11T10:00:00Z",
  "project": "AI研發計畫",
  "summary": "今天主要負責AI研發計畫，完成了模型訓練與部署測試，但遇到的問題是GPU記憶體不足。明天計劃進行LLM退理速度嘗試，用量化技術減少記憶體的使用。"
}
✅ 解析成功： {'employee_id': 'EMP123', 'timestamp': '2025-02-11T10:00:00Z', 'project': 'AI研發計畫', 'summary': '今天主要負責AI研發計畫，完成了模型訓練與部署測試，但遇到的問題是GPU記憶體不足。明天計劃進行LLM退理速度嘗試，用量化技術減少記憶體的使用。'}


In [ ]:
import pandas as pd

# 轉成 DataFrame
df = pd.DataFrame([parsed_output])

# 顯示表格
print(df)

# 存成 CSV
df.to_csv("employee_log_deepseek.csv", index=False)


  employee_id             timestamp project  \
0      EMP123  2025-02-11T10:00:00Z  AI研發計畫   

                                             summary  
0  今天主要負責AI研發計畫，完成了模型訓練與部署測試，但遇到的問題是GPU記憶體不足。明天計劃...  


### phi4:latest

In [37]:
import requests
import json

# 連接到 Ollama 伺服器
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt):
    payload = {
        "model": "phi4:latest",
        "prompt": prompt,
        "stream": False  # 設為 False 讓回應一次返回
    }
    response = requests.post(OLLAMA_URL, json=payload)
    return response.json()["response"]

# 產生 Prompt
prompt = f"""
請根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記，並輸出成 JSON 格式：
---
{transcription["text"]}
---
請以 JSON 格式輸出，包含：
- "employee_id"
- "timestamp"
- "project"
- "summary"
"""

# 呼叫 Ollama
output_text = query_ollama(prompt)
print("LLM 輸出結果：", output_text)

# 解析 JSON，去除 LLM 可能產生的多餘文字
match = re.search(r"\{.*\}|\[.*\]", output_text, re.DOTALL)
if match:
    json_text = match.group(0)  # 提取純 JSON
    try:
        parsed_output = json.loads(json_text)
        print("✅ 解析成功：", parsed_output)
    except json.JSONDecodeError as e:
        print("❌ JSON 解析錯誤：", e)
else:
    print("❌ 解析失敗，請檢查 LLM 輸出")



LLM 輸出結果： 根據提供的員工日誌內容，我們可以從中解析出以下信息：

1. **員工 ID**：來自“我是員工編號EMP123”。
2. **時間戳記**：來自“今天是2025年2月11日早上10點”，所以時間戳記為2025-02-11 10:00。
3. **專案名稱**：來自“我今天主要負責的專案是AI研發計畫”。
4. **摘要**：包含日誌中提到的工作內容和挑戰，例如完成模型訓練與部署測試、遇到的GPU記憶體不足問題以及未來計畫。

以下是以 JSON 格式輸出的結果：

```json
{
  "employee_id": "EMP123",
  "timestamp": "2025-02-11T10:00:00",
  "project": "AI研發計畫",
  "summary": "今天主要負責完成了模型訓練與部署測試，但遇到GPU記憶體不足的問題。明天計劃優化LLM退理速度並使用量化技術減少記憶體使用。"
}
```

此 JSON 包含了所有必要的信息，並用自然語言總結了日誌中的主要內容和計劃。
✅ 解析成功： {'employee_id': 'EMP123', 'timestamp': '2025-02-11T10:00:00', 'project': 'AI研發計畫', 'summary': '今天主要負責完成了模型訓練與部署測試，但遇到GPU記憶體不足的問題。明天計劃優化LLM退理速度並使用量化技術減少記憶體使用。'}


In [38]:
import pandas as pd

# 轉成 DataFrame
df = pd.DataFrame([parsed_output])

# 顯示表格
print(df)

# 存成 CSV
df.to_csv("employee_log_phi.csv", index=False)


  employee_id            timestamp project  \
0      EMP123  2025-02-11T10:00:00  AI研發計畫   

                                             summary  
0  今天主要負責完成了模型訓練與部署測試，但遇到GPU記憶體不足的問題。明天計劃優化LLM退理速...  


## ollama chat

```bash
ollama list
ollama run deepseek-r1:8b
```

```bash
根據以下員工日誌內容，解析出專案名稱、員工 ID、時間戳記、總結，並輸出成 JSON 格式：我是員工編號EMP123今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LLM推理速度嘗試用量化技術減少記憶體的使用
```

# 模型表現

## Phi4:latest

根據提供的員工日誌內容，我們可以解析出以下信息：
- **專案名稱**：AI研發計畫
- **員工 ID**：EMP123
- **時間戳記**：2025年2月11日早上10點
- **總結**：
  - 主要負責的專案是AI研發計畫。
  - 完成了模型訓練與部署測試。
  - 遇到的問題是GPU記憶體不足。
  - 明天的計畫包括優化LLM推理速度，並嘗試用量化技術來減少記憶體使用。

以下是將這些信息以 JSON 格式輸出：

```json
{
  "project_name": "AI研發計畫",
  "employee_id": "EMP123",
  "timestamp": "2025-02-11 10:00",
  "summary": {
    "responsibilities": "主要負責的專案是AI研發計畫。",
    "accomplishments": "完成了模型訓練與部署測試。",
    "challenges": "遇到的問題是GPU記憶體不足。",
    "next_steps": "明天計畫優化LLM推理速度，嘗試用量化技術減少記憶體的使用。"
  }
}
```


## deepseek 8b

<think>
嗯，好的，我需要帮助用户将他们的员工日志信息转化为纯JSON格式，并添加一个摘要。首先，我要确认日志中的内容：

“我是員工編號EMP123今天是2025年2月11日早上10點我今天主要負責的專案是AI研發計畫今天完成了模型訓練與部署測試但遇到的問題是GPU記憶體不足明天計畫優化LLM推理速度嘗試用量化技術減少記憶     
體的使用”

从日志中，我可以提取出以下信息：

- 员工ID：EMP123
- 日期和时间：2025年2月11日早上10點
- 专案名称：AI 研发 计划

在用户之前的查询中，我已经将这些信息转换为JSON格式。现在，用户希望添加一个摘要（summary），因此我需要根据日志内容撰写一个简短的总结。

总结应该包括主要事件、遇到的问题以及下一步计划。具体来说：

1. 员工完成了模型训练和部署测量。
2. 遇到了GPU内存不足的问题。
3. 明天计划优化LLM推理速度，尝试使用量化技术减少内存使用。

基于这些信息，我可以写一个摘要，简要说明当天的工作内容、遇到的挑战以及明天的计划。

接下来，我需要将这些整合成一个JSON对象，并添加summary字段。确保整个结构符合用户要求的纯JSON格式，不使用任何Markdown或其他格式。

```json
{
  "employee_id": "EMP123",
  "project_name": "AI 研发 计划",
  "timestamp": "2025年2月11日早上10點",
  "summary": "完成模型训练和部署测量，遇到GPU内存不足的问题，明天计划优化LLM推理速度并尝试使用量化技术减少内存使用。"
}
```

这样，我就将员工日志信息整理成了一个包含employee_id、project_name、timestamp以及summary的JSON对象。摘要部分简要总结了当天的工作内容和下一步计划，符合用户的要求。

在处理这个问题时，我需要确保不遗漏任何重要信息，同时保持数据的准确性和格式的正确性。此外，还要注意JSON的语法是否正确，避免使用非标准的字符或格式。

总结来说，这个过程包括：

1. 理解用户的需求：提取员工ID、专案名称和时间戳，并添加一个摘要。
2. 分析日志内容，提取所需信息。
3. 撰写摘要，涵盖主要事件和解决问题的计划。
4. 整合所有信息，形成正确的JSON格式。

通过这些步骤，我能够准确地满足用户的要求，并提供一个结构清晰、内容完整的响应。
</think>

```json
{
  "employee_id": "EMP123",
  "project_name": "AI 研发 计划",
  "timestamp": "2025年2月11日早上10點",
  "summary": "完成模型训练和部署测量，遇到GPU内存不足的问题，明天计划优化LLM推理速度并尝试使用量化技术减少内存使用。"
}
```